In [1]:
%matplotlib widget

from mpl_toolkits import mplot3d

import time
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

### Plotting code, not relevant to solver ###
arrow_prop_dict = dict(mutation_scale=20, arrowstyle='->', shrinkA=0, shrinkB=0)
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

def draw_coordinate_frame(ax, T, R):
    tx, ty, tz = T
    new_x = R @ np.array([1, 0, 0.0]) + T
    new_y = R @ np.array([0, 1, 0.0]) + T
    new_z = R @ np.array([0, 0, 1.0]) + T
    
    a = Arrow3D([tx, new_x[0]], [ty, new_x[1]], [tz, new_x[2]], **arrow_prop_dict, color='r')
    ax.add_artist(a)
    a = Arrow3D([tx, new_y[0]], [ty, new_y[1]], [tz, new_y[2]], **arrow_prop_dict, color='b')
    ax.add_artist(a)
    a = Arrow3D([tx, new_z[0]], [ty, new_z[1]], [tz, new_z[2]], **arrow_prop_dict, color='g')
    ax.add_artist(a)
    
def draw_vec(ax, p1, p2, color):
    a = Arrow3D([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], **arrow_prop_dict, color=color)
    ax.add_artist(a)

In [2]:
def project_to_plane(origin, vec):
    ground_normal, ground_point = np.array([0, 0, 1]), np.array([0, 0, 0])
    t = (-ground_normal @ (origin - ground_point)) / (ground_normal @ vec)
    return t

In [32]:
from scipy.spatial.transform import Rotation

data = np.load("drone_target_sample.npz", allow_pickle=True)
x0 = data["drone_x"]  # initial drone state 
T = x0[:3]

q0 = data["drone_q"]
R = Rotation.from_quat(q0)

N = 10
pix_3d = data["pixel_drone"]
target_traj = data["target_traj"]  # N x 3 target positions over horizon of N
print(target_traj.shape)

(10, 3)


In [34]:
import casadi

def norm(vec):
    return (vec[0]**2 + vec[1]**2 + vec[2]**2)**0.5

def state_cost(z, args):
    """
    target_pixel is the 3D coordinate of a pixel in the drone's frame. Needs to be
    transformed into world frame
    """
    xt_target, target_pixel = args[:3], args[3:]
    xt = z[4:]
    u = z[:4]
    yaw = xt[-1]
    # TODO: Check if np and Rotation can handle these variables
    # vel = np.array([xt[3], xt[4], xt[5]])
    # vel = vel / np.linalg.norm(vel)
    # # TODO: check if vel[1] or [2]!!
    # theta = math.asin(-vel[2])  # pitch
    # phi = 0  # cannot be determined from velocity
    # (R @ X + T) - T

    # TODO: convert all to float16
    # try https://cupy.dev/
    # try https://scikit-cuda.readthedocs.io/en/latest/
    # try cython
    # convert python to c+
    rot = np.array([
        [1, 0, 0.0],
        [0, casadi.cos(yaw), -casadi.sin(yaw)],
        [0, casadi.sin(yaw), casadi.cos(yaw)]
    ])
    # desired 3D vector through center pixel of image
    vec_target = casadi.mtimes(rot, target_pixel)
    vec_target = vec_target / norm(vec_target)
    
    # actual 3D vector from drone camera center to target
    vec_cur = xt_target - xt[:3]
    vec_cur = vec_cur / norm(vec_cur)

    # want vectors to align (dot-prod = +1)
    viewpoint_cost = (1 - casadi.mtimes(vec_target.T, vec_cur)) ** 2
    
    # maintain desired height
    desired_height_cost = (xt[0] - 8) ** 2
    
    # maintain desired horiz distance from target
    horiz_dist = (xt[0] - xt_target[0]) ** 2 + (xt[1] - xt_target[1]) ** 2
    desired_horiz_dist = 22
    desired_dist_cost = (desired_horiz_dist - horiz_dist) ** 2
    
    R = np.diag([1, 1, 1, 0.001])
    control_cost = casadi.mtimes([u.T, R, u])
    
    return 100 * viewpoint_cost + 10* desired_height_cost + 10*desired_dist_cost + 0.01 * control_cost

In [68]:
import casadi

# Upper/lower variable bounds lb <= eval_state_constraints(z) <= ub
# https://forces.embotech.com/Documentation/high_level_interface/index.html?highlight=symbolicmodel#inequalities
max_yaw_rate = (2*math.pi / 3) * dt  # max rotate rad/sec * (dt sec / 1 step)
max_horiz_acc = 1 * dt  # m/s^2 * (dt sec / 1 step)
max_vert_acc = 2 * dt  # m/s^2 * (dt sec / 1 step)
# ax  ay  az  yaw_rate  
ctrl_lb = np.array([ -max_horiz_acc, -max_horiz_acc, -max_vert_acc, -max_yaw_rate])
ctrl_ub = np.array([ max_horiz_acc, max_horiz_acc, max_vert_acc, max_yaw_rate])

max_horiz_vel = 5  # m/s
max_vert_vel = 5  # m/s
min_yaw = -np.Inf
max_yaw = np.Inf
min_y = 0 # TODO!!!
max_y = 6
min_x = -6
max_x = 6
min_z = 6
max_z = 10
# x  y  z  vx  vy  vz  yaw
state_lb = np.array([min_x, min_y, min_z, -max_horiz_vel, -max_horiz_vel, -max_vert_vel, min_yaw])
state_ub = np.array([max_x, max_y, max_z, max_horiz_vel, max_horiz_vel, max_vert_vel, max_yaw])

In [92]:
print(np.concatenate([ctrl_lb, state_lb]))
pos_constraints = np.concatenate([ctrl_lb, state_lb])[4:7]  # state_lb[0:3]

[-0.1        -0.1        -0.2        -0.20943951 -6.          0.
  6.         -5.         -5.         -5.                -inf]


In [82]:
# TODO: see if we can position drone to face the correct direction and thus set x/y directions to be straight
# Casadi operations: http://casadi.sourceforge.net/cheatsheets/python.pdf
# def eval_state_constraints(z):
#     rot_align_constraints = np.eye(3)  # TODO (rotate about z-axis)
#     aligned_pos = rot_align_constraints @ z[4:7]  # rotate position
# #     print(aligned_pos.shape, z[:4].shape)
#     return casadi.vertcat(z[:4], aligned_pos, z[7:])

In [95]:
import forcespro.dump
import forcespro.nlp
import sys

# Flat quadrotor dynamics 
dt = 0.1
state_dim = 7
ctrl_dim = 4
A = np.eye(state_dim)
A[0:3, 3:6] = np.eye(3) * dt
B = np.zeros((state_dim, ctrl_dim))
B[:3, :3] = 0.5 * np.eye(3) * dt ** 2
B[3:, :] = np.eye(4) * dt


model = forcespro.nlp.SymbolicModel()  # create empty model
model.N = N  # horizon length
model.nvar = 4 + 7  # number of variables
model.neq = 7  # number of equality constraints
model.npar = 6 # number of runtime parameters
model.objective = state_cost
model.objectiveN = state_cost
model.eq = lambda z: A @ z[4:] + B @ z[:4]
model.E = np.concatenate([np.zeros((7,4)), np.eye(7)], axis=1)

# State and contrl constraints
# model.hl = np.concatenate([ctrl_lb, state_lb])
# model.hu = np.concatenate([ctrl_ub, state_ub])
# model.ineq = eval_state_constraints
model.lb = np.concatenate([ctrl_lb, state_lb])
model.ub = np.concatenate([ctrl_ub, state_ub])

model.xinitidx = range(4,11) 

codeoptions = forcespro.CodeOptions('FORCESNLPsolver')
codeoptions.maxit = 1000     # Maximum number of iterations
codeoptions.printlevel = 0  # Use printlevel = 2 to print progress (but 
#                             not for timings)
codeoptions.optlevel = 0    # 0 no optimization, 1 optimize for size, 
#                             2 optimize for speed, 3 optimize for size & speed
codeoptions.cleanup = False
codeoptions.timing = 1
# codeoptions.nlp.hessian_approximation = 'bfgs'
# codeoptions.solvemethod = 'PDIP_NLP' # choose the solver method Sequential 
#                              Quadratic Programming
# codeoptions.sqp_nlp.maxqps = 1      # maximum number of quadratic problems to be solved
# codeoptions.sqp_nlp.reg_hessian = 5e-9 # increase this if exitflag=-8
codeoptions.nlp.bfgs_init = 3.0*np.identity(7) # initialization of the hessian approximation
# codeoptions.noVariableElimination = 1

# debugging 
tag,full_filename = forcespro.dump.save_formulation(model, codeoptions)

solver = model.generate_solver(options=codeoptions)


This is FORCESPRO v4.2.1, a code generator for fast numerical optimization.
Copyright (C) embotech AG [info@embotech.com], 2013-2021. All rights reserved.
Warning. Host platform supports FMA instructions, hence code-generation option fma_supported is set.
Using server https://forces.embotech.com.


Variables 4 through 10 in stage 0 will be eliminated. Please check your nonlinear inequality constraints to make sure they remain valid. To prevent variable elimination use codeoptions.noVariableElimination = 1.


Server is running version v4.2.1                        [OK]
Preparing data to be sent...                            [OK]
Generating and compiling code...                        [OK]
Code successfully generated in 22.71 sec.
Package downloaded to FORCESNLPsolver.zip.
Directory /home/alvin/drone_ws/src/hawkeye/scripts/FORCESNLPsolver/ will be overwritten. Proceed? [y]/n y
Extracting package...                                   [OK]

Code generation successfully completed. Happy solving!

Your downloaded files contain the following directories:
- include: C header files
- interface: Matlab, Simulink and Python interfaces for your solver on the host machine
- obj: solver object files for simulation on Linux host machine
- lib: solver library files for simulation on Linux host machine

Import the interface module FORCESNLPsolver_py and type
    help(FORCESNLPsolver_py)
for help on how to call the solver from a Python script.

 


In [99]:
full_filename

'FORCESNLPsolver_LSE8O13AWO0_20214261230569592_F.json'

In [96]:
# https://forces.embotech.com/Documentation/solver_options/index.html?highlight=pdip_nlp
# Set initial condition
z0i = np.zeros((model.nvar,1))
z0 = np.transpose(np.tile(z0i, (1, model.N)))
problem = {"x0": z0,
           "xinit": x0}

# Set runtime parameters (here, the next N points on the path)
horizon_inputs = np.zeros((model.N, model.npar))
horizon_inputs[:, 3:] = pix_3d
horizon_inputs[:, :3] = target_traj
problem["all_parameters"] = np.reshape(horizon_inputs, (-1, 1))

# Time to solve the NLP!
print(dir(solver))
output, exitflag, info = solver.solve(problem)

# Make sure the solver has exited properly.
assert exitflag == 1, "bad exitflag"
sys.stderr.write("FORCESPRO took {} iterations and {} seconds to solve the problem.\n"\
    .format(info.it, info.solvetime))

# Extract output
temp = np.zeros((model.N, model.nvar))
for i in range(0, model.N):
    temp[i, :] = output['x{0:02d}'.format(i+1)]
U_mpc = temp[:, :4]
X_mpc = temp[:, 4:]

['_Solver__is_external_file', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_info_struct_ctype', '_is_set_up', '_outputs_struct_ctype', '_params_struct_ctype', '_solver_func_ctype', 'add_output', 'add_param', 'build', 'c_headers', 'create_ctypes', 'dynamic_locally_built', 'f_eval', 'f_eval_proto', 'f_solver', 'from_definitions_file', 'from_directory', 'help', 'info_decls', 'intel_libs_dir', 'is_convex', 'is_static', 'lib_eval', 'lib_solver', 'lib_solver_static', 'load_f_eval', 'load_f_solver', 'name', 'output_decls', 'param_decls', 'preload_dynamic_libraries', 'remove_param', 'root', 'setup', 'solve', 'static_locally_built', 'store_eval_definitions']


FORCESPRO took 112 iterations and 0.019605145 seconds to solve the problem.


In [97]:
X_mpc[:, :3]

array([[ 0.02089381,  0.02530667,  7.99154234],
       [-0.39903904,  0.17732435,  7.8673352 ],
       [-1.42638341,  0.69195646,  7.52305741],
       [-2.66432805,  1.61698595,  7.00307621],
       [-3.83561997,  2.88422384,  6.34260275],
       [-4.78033956,  4.34621561,  5.57106369],
       [-5.4478851 ,  5.83024654,  4.71498919],
       [-5.87766227,  7.19875055,  3.79963803],
       [-6.160386  ,  8.39998019,  2.84891699],
       [-6.38471123,  9.48395382,  1.8829926 ]])

In [98]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
target_x = [data["target_traj"][i][0] for i in range(len(data["target_traj"]))]
target_y = [data["target_traj"][i][1] for i in range(len(data["target_traj"]))]
target_z = [data["target_traj"][i][2] for i in range(len(data["target_traj"]))]
ax.plot(target_x, target_y, target_z, label='target path')
for target_pos in data["target_traj"]:
    draw_coordinate_frame(ax, T=target_pos, R=np.eye(3))
    
ax.plot(X_mpc[:,0], X_mpc[:,1], X_mpc[:,2], label='drone path')
for i in range(X_mpc.shape[0]):
    rot = Rotation.from_euler("ZYX", [X_mpc[i, -1], 0, 0]).as_matrix()
    draw_coordinate_frame(ax, T=X_mpc[i, :3], R=rot)
    desired_vec = rot @ pix_3d
    t = project_to_plane(X_mpc[i, :3], desired_vec)
    draw_vec(ax, X_mpc[i, :3], desired_vec*t + X_mpc[i, :3], color='g')
draw_coordinate_frame(ax, T=T, R=R.as_matrix())

# vec from drone to target
true_vec = data["target_traj"][0] - x0[:3]
draw_vec(ax, x0[:3], data["target_traj"][0], color='r')

# desired vec (center of image plane)
yaw = x0[-1]
vel = np.array([x0[3], x0[4], x0[5]])
vel = vel / np.linalg.norm(vel)
theta = math.asin(-vel[1])  # pitch
print([yaw, theta, phi])
print(R.as_euler("ZYX"))
fake_R = Rotation.from_euler('ZYX', [yaw, 0, 0])
desired_vec = fake_R.as_matrix() @ pix_3d
t = project_to_plane(T, desired_vec)
draw_vec(ax, T, desired_vec*t + T, color='g')

ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0.7883263604479005, 0.46650359121892887, 0.0010596015757493582]
[7.88327183e-01 1.62825303e-04 1.66669236e-03]
